In [0]:
%sql
SELECT 
  value,
  regexp_replace(
    value, 
    '"order_date": (\\d{4}-\\d{2}-\\d{2})', 
    '"order_date": "\$1"'
  ) AS fixed_value
FROM gizmo_box.bronze.v_orders;


In [0]:
%sql 

CREATE OR REPLACE TEMP VIEW tv_orders_fixed as 
SELECT 
  value,
  regexp_replace(
    value, 
    '"order_date": (\\d{4}-\\d{2}-\\d{2})', 
    '"order_date": "\$1"'
  ) AS fixed_value
FROM gizmo_box.bronze.v_orders;


In [0]:
%sql
-- Pulling schema from json
select schema_of_json(fixed_value) from tv_orders_fixed
limit 1;

In [0]:
%sql
-- Pulling schema from json
select from_json(fixed_value, 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as json_value from tv_orders_fixed

In [0]:
%sql
DROP TABLE IF EXISTS gizmo_box.silver.orders;
CREATE OR REPLACE TABLE gizmo_box.silver.orders_json AS

select from_json(fixed_value, 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as json_value from tv_orders_fixed

In [0]:
%sql
select * from gizmo_box.silver.orders_json;